In [ ]:
import tensorflow as tf
import json
import numpy as np
import pandas as pd

from google.cloud import bigquery
from google.cloud import storage

from google.cloud import aiplatform


import matplotlib.pyplot as plt
import seaborn as sns
#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom-tabular-bq-managed-dataset.ipynb

In [ ]:
print(tf.__version__)

In [ ]:
#table_id='new_incident'
table_id='test_incident'
# table_id='validation_incident'

projectId='pongthorn'
dataset_id='DemoSMartDW'
REGION="asia-southeast1"

if  table_id=='new_incident':
 unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']
 filePath='../../data/Bi_Prediction_New_Incident.csv'
elif table_id=='test_incident' or table_id=='validation_incident':
 unUsedColtoPredict=['severity','id','severity_id','severity_name']  
 filePath=f'../../data/Bi_Prediction_{table_id}.csv'
    
isLocalModel=True

isBQToCSV=True

PATH_FOLDER_ARTIFACTS="model_binary"  

modelFile_endPoint=1 
#1=local(dev)/gcs(clound function) and 2=vertext

localFile=True


In [ ]:
# from google.oauth2 import service_account
# mycredentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')

if isBQToCSV:
    
    bqclient= bigquery.Client(project=projectId)
    # bqclient= bigquery.Client(project=projectId,credentials=mycredentials)
    def download_table(bq_table_uri: str):

        prefix = "bq://"
        if bq_table_uri.startswith(prefix):
            bq_table_uri = bq_table_uri[len(prefix) :]

        table = bigquery.TableReference.from_string(bq_table_uri)
        rows = bqclient.list_rows(table)
        return rows.to_dataframe()

    dfNewData=download_table(f"{projectId}.{dataset_id}.{table_id}")
    dfNewData.to_csv(filePath,index=False)    
    print(dfNewData.info())


# Load and Map Data Test

In [ ]:
if  localFile:
    dfNewData=pd.read_csv(filePath)

# dfNewData=dfNewData.sample(3)

In [ ]:
mapping_file="incident_severity_to_binary.json"

with open(mapping_file, 'r') as json_file:
 map_sevirity_to_class= json.load(json_file)
                                  
print(map_sevirity_to_class)


In [ ]:
if table_id=='new_incident':
    dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)
    print(dfNewData.info())
elif table_id=='test_incident' or table_id=='validation_incident':
    
    dfNewData=dfNewData.rename(columns={'label_binary_severity':'severity'})
    dfNewData=dfNewData.drop(columns=['label_multi_severity'])
    if 'id' not in dfNewData.columns.to_list():
        dfNewData=dfNewData.reset_index(drop=True)
        dfNewData = dfNewData.reset_index(level=0)
        dfNewData.rename(columns={"index": "id"},inplace=True)
        dfNewData['id']=dfNewData['id']+1
print(filePath)  

# dfNewData=dfNewData.sample(10)
print(dfNewData.info())
dfNewData.tail(10)


In [ ]:
def plot_class(df):
    fig , ax = plt.subplots(figsize=(8,5))
    ax =sns.countplot(x='severity', data=df,)
    for p in ax.patches:
       ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01))
    plt.title('Severity')
    plt.show()
    
plot_class(dfNewData)

# Load Model from Directory to Predict

In [ ]:
model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
print(f"Load from {PATH_FOLDER_ARTIFACTS}")
# model.summary()

In [ ]:
pdPrediction=pd.DataFrame(columns=['_id','_predict_severity'])

for  row_dict in dfNewData.to_dict(orient="records"):
      incident_id=row_dict['id']
      print(f"{incident_id} - {row_dict['severity']}") 
      for key_removed in unUsedColtoPredict:
       row_dict.pop(key_removed)
      # print(row_dict)  

      input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}
      predictionResult = model.predict(input_dict)

      prob = tf.nn.sigmoid(predictionResult[0])
      prob_pct=(100 * prob)   
      _class= 1 if prob[0]>=0.5 else 0  
      print(f"{prob_pct[0]} % at {_class} as 0(Normal)/1(Critcal)") # np.argmax(prob, axis=0
      
      
      pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([{'_id':incident_id, '_predict_severity':_class}])] )
      # print(input_dict)  
   
      print("======================================================================================")

dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
dfPredictData=dfPredictData.drop(columns=['_id'])
dfPredictData=dfPredictData[['id','_predict_severity','severity']]
dfPredictData

# Classification Rerport

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
className=list(set().union(list(dfPredictData['severity'].unique()),list(dfPredictData['_predict_severity'].unique())))
actualClass=[  f'actual-{x}' for x in  className]
predictedlClass=[  f'predcited-{x}' for x in className]

y_true=list(dfPredictData['severity'])
y_pred=list(dfPredictData['_predict_severity'])
cnf_matrix = confusion_matrix(y_true,y_pred)
cnf_matrix

# #index=actual , column=prediction
cm_df = pd.DataFrame(cnf_matrix,
                     index = actualClass, 
                     columns = predictedlClass)
cm_df

In [ ]:
print(classification_report(y_true, y_pred, labels=className))